In [4]:
from tensorflow.keras import Input
from tensorflow.keras import layers, Model

input_img = Input(shape=(600, 600, 3))

print(input_img)
x = layers.Conv2D(512, kernel_size = 3, strides = 1, activation='relu', padding='same')(input_img)
print(x)
x = layers.Conv2D(512, kernel_size = 3, strides = 1, activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
print(x)
x = layers.Conv2D(256, kernel_size = 3, strides = 1, activation='relu', padding='same')(input_img)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
print(x)

x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = layers.MaxPooling2D((2, 2), padding='same')(x)
print(x)
encoder = Model(input_img, x)


x = layers.Conv2DTranspose(256, (3, 3), strides = 2, activation='relu', padding='same')(x)
print(x)
x = layers.Conv2DTranspose(512, (3, 3), strides = 1, activation='relu', padding='same')(x)
print(x)
x = layers.Conv2DTranspose(512, (3, 3), strides = 2, activation='relu', padding='same')(x)
print(x)
x = layers.Conv2DTranspose(3, (3, 3), strides = 1, activation='relu', padding='same')(x)
print(x)
decoder = Model(input_img,x)

Tensor("input_2:0", shape=(None, 600, 600, 3), dtype=float32)
Tensor("conv2d/Identity:0", shape=(None, 600, 600, 512), dtype=float32)
Tensor("max_pooling2d/Identity:0", shape=(None, 300, 300, 512), dtype=float32)
Tensor("max_pooling2d_1/Identity:0", shape=(None, 300, 300, 256), dtype=float32)
Tensor("max_pooling2d_2/Identity:0", shape=(None, 150, 150, 256), dtype=float32)
Tensor("conv2d_transpose/Identity:0", shape=(None, 300, 300, 256), dtype=float32)
Tensor("conv2d_transpose_1/Identity:0", shape=(None, 300, 300, 512), dtype=float32)
Tensor("conv2d_transpose_2/Identity:0", shape=(None, 600, 600, 512), dtype=float32)
Tensor("conv2d_transpose_3/Identity:0", shape=(None, 600, 600, 3), dtype=float32)


In [ ]:
import numpy as np
a = np.zeros([1,600,600,3])
a.shape

In [ ]:
encoder.predict(a).shape, decoder.predict(a).shape

In [ ]:
from tensorflow.keras import backend as K
K.clear_session()

In [1]:
from Models.models import dn121scratch as dns

In [2]:
m = dns()

29089792/29084464 [==============================] - 8s 0us/step


In [3]:
m.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
densenet121 (Model)          (None, 1024)              7037504   
_________________________________________________________________
dense (Dense)                (None, 1)                 1025      
Total params: 7,038,529
Trainable params: 6,954,881
Non-trainable params: 83,648
_________________________________________________________________


In [ ]:
import tensorflow as tf
tf.__version__[0]

In [5]:
decoder.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 600, 600, 3)]     0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 600, 600, 256)     7168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 300, 300, 256)     0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 300, 300, 256)     590080    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 150, 150, 256)     0         
_________________________________________________________________
conv2d_transpose (Conv2DTran (None, 300, 300, 256)     590080    
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 300, 300, 512)     1180